### Performance reports

In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
# preparation database
import sqlite3

txhash_path = 'txhash.db'
conn = sqlite3.connect(txhash_path)
cur = conn.cursor()
index_path = './completed/index.db'
iconn = sqlite3.connect(index_path)
icur = iconn.cursor()

def create_meta_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Meta (
                     key TEXT PRIMARY KEY,
                     value TEXT);
                ''')

def create_txin_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxIn (
                     tx INTEGER,
                     addr INTEGER);
                ''')

def create_txout_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxOut (
                     tx INTEGER,
                     addr INTEGER);
                ''')

def insert_txin(tx, addr):
    cur.execute('''INSERT OR IGNORE INTO TxIn (
                       tx, addr) VALUES (
                       ?, ?);
                ''', (tx, addr))
    
def insert_txout(tx, addr):
    cur.execute('''INSERT OR IGNORE INTO TxOut (
                       tx, addr) VALUES (
                       ?, ?);
                ''', (tx, addr))

def update_meta(key, value):
    cur.execute('''INSERT OR IGNORE INTO Meta (
                        key, value) VALUES (
                        ?, ?);
                ''', (key, value))
    cur.execute('''UPDATE Meta SET value = ? WHERE key = ?;
                ''', (key, value))

def get_txid(txhash):
    icur.execute('''SELECT id FROM TxID WHERE txhash = ?''', (txhash,))
    return icur.fetchone()[0]
    
def get_addrid(addr):
    icur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (addr,))
    return icur.fetchone()[0]
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def get_meta(key):
    cur.execute('''SELECT value FROM Meta WHERE key = ?''', (key,))
    result = cur.fetchone()
    if result is not None:
        result = result[0]
    return result

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
create_meta_table()
create_txin_table()
create_txout_table()

In [ ]:
def get_addresses(theight):
    iaddresses = list()
    oaddresses = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    for tx in block['tx']:
        iaddrs = list()
        oaddrs = list()
        for vin in tx['vin']:
            if 'coinbase' in vin.keys():
                iaddrs.extend(['coinbase'])
                continue
            ptx = rpc_connection.getrawtransaction(vin['txid'], 1)
            pvout = ptx['vout'][vin['vout']]
            if pvout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                 'witness_v0_keyhash', 'witness_v0_scripthash',
                                                 'witness_unknown', 'multisig'):
                iaddrs.extend(pvout['scriptPubKey']['addresses'])
            elif pvout['scriptPubKey']['type'] in ('pubkey', 'nonstandard'):
                try:
                    iaddrs.extend([pubkey_to_address(get_pubkey(pvout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    iaddrs.extend([f'{vin["txid"]}{vin["vout"]}'])
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vin}')
        n = -1
        for vout in tx['vout']:
            n = n + 1
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'witness_unknown', 'multisig'):
                oaddrs.extend(vout['scriptPubKey']['addresses'])
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard'):
                try:
                    oaddrs.extend([pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    oaddrs.extend([f'{tx["hash"]}{n}'])
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vout}')
        txid = get_txid(tx['hash'])
        for addr in set(iaddrs):
            if addr == 'coinbase':
                addrid = 0
            else:
                addrid = get_addrid(addr)
            iaddresses.append((txid, addrid))
        for addr in set(oaddrs):
            if addr == 'coinbase':
                addrid = 0
            else:
                addrid = get_addrid(addr)
            oaddresses.append((txid, addrid))
    return iaddresses, oaddresses

In [ ]:
import time
import pickle
import itertools
import multiprocessing

if os.path.exists('taking_txhash.pickle'):
    with open('taking_txhash.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = dict()

term = 1000
start_height = get_meta('height')
if start_height is None or start_height < term:
    start_height = 0
else:
    start_height = start_height - term
end_height = best_block['height']-term # default
print(f'Start from {start_height} to {end_height}')

rpc_connection = get_rpc()
pool_num = multiprocessing.cpu_count()
for sheight, eheight in zip(range(start_height, end_height, term), range(start_height+term, end_height+term, term)):
    stime = time.time()
    begin_transactions()
    if eheight >= end_height:
        eheight = end_height+1
    with multiprocessing.Pool(pool_num) as p:
        results = p.imap(get_addresses, range(sheight, eheight))
        for iaddresses, oaddresses in results:
            for txid, addrid in iaddresses:
                insert_txin(txid, addrid)
            for txid, addrid in oaddresses:
                insert_txout(txid, addrid)
    update_meta('height', eheight-1)
    commit_transactions()
    etime = time.time()
    taking[sheight] = etime-stime
    print(f'Job done from {sheight} to {eheight} during {etime-stime}')

In [ ]:
with open('taking_txhash.pickle', 'wb') as f:
    pickle.dump(taking, f)
conn.close()
iconn.close()